In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [2]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')

In [5]:
driver.find_element('id','username').send_keys('matheus.ph.silva.23@gmail.com')
driver.find_element('id','password').send_keys('ufmg2024')
driver.find_element(By.XPATH, "//button").click()

In [26]:
driver.get('https://www.linkedin.com/search/results/people/?geoUrn=%5B%22106057199%22%5D&keywords=software%20developer&network=%5B%22O%22%5D&origin=FACETED_SEARCH&pastCompany=%5B%224206%22%5D&sid=tdI')

In [27]:
page= 1
publicProfilesUrl = []
print(len(publicProfilesUrl))
while True:
    print(page)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    htmlPage = driver.page_source
    site = BeautifulSoup(htmlPage)
    profiles=site.find_all("li","reusable-search__result-container")
    for profile in profiles:
        if(len(profile.find_all(string="LinkedIn Member"))==0):
            publicProfilesUrl.append(profile.find('a','app-aware-link')['href'])
    page+=1
    goToNextPage = not site.find("button", "artdeco-pagination__button--next").has_attr('disabled')
    if not goToNextPage : 
        break
    driver.find_element(By.CSS_SELECTOR, ".artdeco-pagination__button--next").click()
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.reusable-search__result-container'))
    WebDriverWait(driver, 10).until(element_present)

print(len(publicProfilesUrl))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
667


In [28]:
profiles_schema = pa.schema([
('url', pa.string()),
])
paProfiles = pa.array(publicProfilesUrl)
batch = pa.RecordBatch.from_arrays(
    [paProfiles],
    names=profiles_schema.names
)
table = pa.Table.from_batches([batch])
print(len(paProfiles))
pq.write_table(table, './profiles-url-vivo.parquet')

667


In [29]:
df = pq.read_table(source='./profiles-url-vivo.parquet').to_pandas()
print(df)
print(df['url'][1])

                                                   url
0    https://www.linkedin.com/in/fabio-elias-filpi-...
1    https://www.linkedin.com/in/alanpfabricio?mini...
2    https://www.linkedin.com/in/jorge-santoro-230a...
3    https://www.linkedin.com/in/gabrielcapanemahor...
4    https://www.linkedin.com/in/ppederiva?miniProf...
..                                                 ...
662  https://www.linkedin.com/in/daniloalves?miniPr...
663  https://www.linkedin.com/in/maria-santana-b847...
664  https://www.linkedin.com/in/marceloabo?miniPro...
665  https://www.linkedin.com/in/ronaldo-riccio-dos...
666  https://www.linkedin.com/in/fabiane-frogeri-64...

[667 rows x 1 columns]
https://www.linkedin.com/in/alanpfabricio?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABKz900BG3pNaqN2GXpKjKPOXvF11uY1wx8


https://www.linkedin.com/in/pedrohenrique-almeida47?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABvU6Z4BMcppz-B4GigCajjf3c8nkV_hgvY
